In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
url = 'https://www.cars.com/shopping/results/?dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=&maximum_distance=150&mileage_max=100000&page_size=100&sort=best_match_desc&stock_type=all&year_max=&year_min=&zip=55423'
page = requests.get(url)
page

<Response [200]>

In [3]:
soup = BeautifulSoup(page.text, 'lxml')

In [4]:
columns = ['Year','Make','Model','Used/New','Price','ConsumerRating','ConsumerReviews','SellerType','SellerName','SellerRating','SellerReviews',
           'StreetName','State','Zipcode','DealType','ComfortRating','InteriorDesignRating','PerformanceRating','ValueForMoneyRating',
           'ExteriorStylingRating','ReliabilityRating','ExteriorColor','InteriorColor','Drivetrain','MinMPG','MaxMPG','FuelType',
           'Transmission','Engine','VIN','Stock#','Mileage']   

df= pd.DataFrame(columns=columns)

In [5]:
for i in range(12):
    
    postings = soup.find_all('a', class_ = 'vehicle-card-link js-gallery-click-link')
    current_page = soup
    for post in postings:
        try:
            link = 'http://cars.com' + post.get('href')
            info_page = requests.get(link)
            soup = BeautifulSoup(info_page.text, 'lxml')
            
            ###DELETE THIS WHEN YOURE DONE
            #test_link = 'https://www.cars.com/vehicledetail/1d26a033-1886-4e9c-80cb-fb4f15ff4d51/'
            #info_page = requests.get(test_link)
            #soup = BeautifulSoup(info_page.text, 'lxml')
            
            
            #Data we want
            title_info = soup.find('h1', class_ = 'listing-title').text.split()
            year = title_info[0]
            make = title_info[1]
            model = ' '.join(title_info[2:])
            used_or_new = soup.find('p', class_ = 'new-used').text
            price = soup.find('span', class_ = 'primary-price').text
            consumer_rating = float(soup.find('section', class_= 'sds-page-section vehicle-reviews').find('span', class_='sds-rating__count').text)
            consumer_reviews = int(soup.find('section',class_= 'sds-page-section vehicle-reviews').find('a',class_= 'sds-rating__link sds-button-link').text[1:].split()[0])
            
            #Seller Details
            #Dealer
            try:
                seller_name = soup.find('h3', class_ = 'sds-heading--5 heading seller-name').text
                seller_rating = float(soup.find('section',class_ = 'sds-page-section seller-info').find('span', class_ = 'sds-rating__count').text)
                seller_reviews = soup.find('a', class_ = 'sds-rating__link sds-button-link').text.split()[0][1:]
                seller_type = 'Dealer'
                if ',' in seller_reviews:
                    seller_reviews = int(''.join(seller_reviews.split(',')))
                else:
                    seller_reviews =  int(seller_reviews)
                address_pieces = soup.find('div', class_ = 'dealer-address').text.split(',')
                street_name = address_pieces[0]
                state = address_pieces[1].split()[0]
                zipcode = address_pieces[1].split()[1] 
                
            #Private seller    
            except:
                seller_type = 'Private'
                pieces = soup.find('div', class_ = 'seller-address').text.split('from')
                seller_name = pieces[0].strip()
                address_pieces = pieces[1].split(',')
                street_name = address_pieces[0].strip()
                state = address_pieces[1].split()[0]
                zipcode = address_pieces[1].split()[1] 
    
            
            #Deal type
            deal_type = 'NA'
            first_badge = soup.find('span', class_ = 'sds-badge__label').text
            if 'deal' in first_badge.casefold():
                deal_type = first_badge.split()[0]
            
            #Rating breakdown
            rb = soup.find_all('div', class_ = 'review-breakdown')[0].find_all('span', class_ = 'sds-definition-list__value')
            
            comfort = float(rb[0].text)
            interior_design = float(rb[1].text)
            performance = float(rb[2].text) 
            value_for_money = float(rb[3].text)
            exterior_styling = float(rb[4].text)
            reliability = float(rb[5].text)
            
            #Basic Description Table
            bd = soup.find_all('div', class_ = 'basics-content-wrapper')[0].find_all('dd')
            
            exterior_color = bd[0].text.strip()
            interior_color = bd[1].text.strip()
            drive_train = bd[2].text.strip()
            min_mpg = int(bd[3].text.split('\n')[2].split('–')[0])
            max_mpg = int(bd[3].text.split('\n')[2].split('–')[1])
            fuel_type = bd[4].text.strip()
            transmission = bd[5].text.strip()
            engine = bd[6].text.strip()
            vin = bd[7].text.strip()
            stock_number = bd[8].text.strip()
            mileage = int(''.join(bd[9].text.split()[0].split(',')))
            
            
    
            
            
            
            new_row = pd.DataFrame([[year,make,model,used_or_new,price,consumer_rating,consumer_reviews,seller_type,seller_name,seller_rating,seller_reviews,
                                   street_name,state,zipcode,deal_type,comfort,interior_design,performance,value_for_money,exterior_styling,reliability,
                                   exterior_color,interior_color,drive_train,min_mpg,max_mpg,fuel_type,transmission,engine,vin,stock_number,mileage]],
                                  columns=columns)
            
            #Generate row in dataframe based on scraped information
            df = pd.concat([df,new_row])
        except:
            pass
    
    current_page    
    current_page.find('a', {'aria-label':'Next page'}).get('href')
    next_page = 'http://cars.com' + current_page.find('a', {'aria-label':'Next page'}).get('href') #Get link to next page
        
    url = next_page
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    print(i)

#file = 'C:/Users/leave/OneDrive/Documents/Scraped_Datasets/Cars_Data/cars_raw.csv'
file = 'C:/Users/John/Desktop/BOOTCAMP/Car_Search_Project'

#df.to_csv(file, index= False)
df.to_excel("cars_raw.xlsx") 

0
1
2
3
4
5
6
7
8
9
10
11


In [6]:
df

,Year,Make,Model,Used/New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerName,SellerRating,...,InteriorColor,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Engine,VIN,Stock#,Mileage
0,2019,BMW,X5 xDrive40i,Used,"$54,190",4.7,55,Dealer,Motorwerks BMW,4.7,...,Amarone,All-wheel Drive,20,26,Gasoline,8-Speed Automatic,3.0L I6 24V GDI DOHC Turbo,5UXCR6C50KLL10954,BN09189T,32415
0,2020,Jeep,Grand Cherokee Limited,Used,"$37,991",4.8,150,Dealer,Bloomington Chrysler Jeep Dodge RAM,4.7,...,Black,Four-wheel Drive,18,25,Gasoline,Automatic,Regular Unleaded V-6 3.6 L/220,1C4RJFBG8LC383724,11AT120P,30030
0,2019,Buick,Encore Essence,Used,"$24,599",4.7,182,Dealer,Walser Buick GMC Bloomington,4.6,...,Ebony,All-wheel Drive,25,30,Gasoline,Automatic,Turbocharged I4 1.4/83,KL4CJGSM7KB795164,4AK391L,28557
0,2022,Genesis,GV70 3.5T Sport,Used,"$70,490",5.0,2,Dealer,Luther Bloomington Hyundai,4.7,...,Red,All-wheel Drive,19,25,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC Twin Turbo,KMUMCDTCXNU040735,2344A,12190
0,2022,Mercedes-Benz,GLB 250 Base 4MATIC,Used,"$50,080",4.9,4,Dealer,Feldmann Imports,4.8,...,Black,All-wheel Drive,23,31,Gasoline,8-Speed Automatic with Auto-Shift,2.0L I4 16V GDI DOHC Turbo,W1N4M4HB3NW186598,B2431,5591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2018,Chevrolet,Equinox 1LT,Used,"$24,995",4.7,708,Dealer,Nyhus Chevrolet Buick Inc,4.0,...,Jet Black,All-wheel Drive,24,30,Gasoline,6-Speed Automatic,1.5L I4 16V GDI DOHC Turbo,2GNAXSEV7J6281908,J6281905,34746
0,2020,Ford,Escape SEL,Used,"$31,594",4.6,230,Dealer,Pischke Motors Chrysler Dodge Fiat Ram Jeep,4.8,...,Ebony,All-wheel Drive,26,31,Gasoline,8-Speed Automatic,1.5L I3 12V GDI DOHC Turbo,1FMCU9H67LUA56797,2822051,31439
0,2011,Ford,F-150 Lariat,Used,"$22,950",4.5,262,Dealer,Mike Molstead Motors,4.7,...,–,Four-wheel Drive,14,19,E85 Flex Fuel,6-Speed Automatic,5.0L V8 32V MPFI DOHC Flexible Fuel,1FTFW1EF1BKD91680,M5587A,72991
0,2021,Jeep,Cherokee Latitude Lux,Used,"$31,924",4.9,71,Dealer,Valu Ford Chrysler,4.8,...,Black,Four-wheel Drive,19,27,Gasoline,9-Speed Automatic,3.2L V6 24V MPFI DOHC,1C4PJMMX4MD158947,F158947,24487
